In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Lab 3.1- Run LandTrendr over Composites

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/3.1-LandTrendr.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/3.1-LandTrendr.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/3.1-LandTrendr.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview

LandTrendr, short for Landsat-based detection of Trends in Disturbance and Recovery, is a temporal segmentation algorithm used for change detection as well as time-series smoothing. You can use LandTrendr outputs for stand-alone change detection or as inputs to modeling models such as the ones used by LCMS. 

**This notebook uses Landsat and Sentinel-2 composites, like the ones you created in the previous notebook, as inputs to the LandTrendr temporal segmentation and spectral smoothing operations.**

##### To learn more about LandTrendr:
- [LandTrendr Guide](https://emapr.github.io/LT-GEE/)
- [LandTrendr Original Publication](https://www.sciencedirect.com/science/article/abs/pii/S0034425710002245)
- [LandTrendr GEE Publication](https://www.mdpi.com/2072-4292/10/5/691)


### Objective

In this tutorial, you learn how to create and manipulate LandTrendr outputs.

**This tutorial uses the following Google Cloud services:**

- Google Earth Engine

**The steps performed include:**

- Create LandTrendr outputs
- Manipulate EE array image objects to get meaningful data out of LandTrendr outputs

**Learning objectives include:**
- Users will understand the purpose of temporal segmentation as implemented in LandTrendr.
- Users will understand key parameters in LandTrendr algorithm. 
- Users will be able to generate and manipulate array outputs from LandTrendr. 


## Before you begin

### If you are working in Workbench: Set your current URL under `workbench_url`
This gives the Map Viewer a url in which to host the viewer we will be generating. 
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/` (See the image below)

![workspace url](img/workspace-url.png)

### Optional: Set a folder to use for all exports under `export_path_root` 
**If you accessed this lab through Google Qwiklabs, do not change `export_path_root`.**
* This folder should be an assets folder in an existing GEE project.
* By default, this folder is the same as the 'pre-baked folder'-- a directory where the outputs have already been created. 
* If you would like to create your own outputs, specify a different path for `export_path_root`, but leave the `pre_baked_path_root` as it was. This way, the pre-baked outputs can be shown at the end, instead of waiting for all exports to finish.
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [26]:
workbench_url = 'https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/'
pre_baked_path_root  = 'projects/rcr-gee/assets/lcms-training'
export_path_root = pre_baked_path_root
print('Done')

Done


### Installation
First, install necessary Python packages. Uncomment the first line to upgrade geeViz if necessary.

Note that for this module, we're also importing the `geeViz.changeDetectionLib as changeDetectionLib`. We will use this library later to deploy the LandTrendr functions.

In [27]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import geeViz.gee2Pandas as g2p

import inspect

ee = getImagesLib.ee
Map = getImagesLib.Map

print('Done')

Done


### Set up your work environment

Specify an image collection path where composites were exported to. In addition, create a blank image collection where your LandTrendr outputs will be exported to.

Currently, when running within Colab or Workbench, geeView uses a different project to authenticate through, so you may need to make your asset public to view from within Colab.

In [28]:
# Create folder and a collection and make public

export_composite_collection = f'{export_path_root}/lcms-training_module-2_composites'
export_landTrendr_collection = f'{export_path_root}/lcms-training_module-3_landTrendr'

aml.create_asset(export_landTrendr_collection,asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_landTrendr_collection,writers = [],all_users_can_read = True,readers = [])

print('Done')

Found the following sub directories:  ['lcms-training', 'lcms-training_module-3_landTrendr']
Will attempt to create them if they do not exist
Asset projects/rcr-gee/assets/lcms-training already exists
Asset projects/rcr-gee/assets/lcms-training/lcms-training_module-3_landTrendr already exists
Updating permissions for:  projects/rcr-gee/assets/lcms-training/lcms-training_module-3_landTrendr
Done


### Set up the map 

Run the code block below to set up the map. If, you notice that the map is still displaying outputs from a previous lab, you may need to re-run this code block to reset the port used for the proxy URL. 

In [48]:
# Set up the map

Map.clearMap()
Map.port = 1235 # reset port if necessary
Map.proxy_url = workbench_url

## Change detection with LandTrendr

LandTrendr is a temporal segmentation algorithm. LandTrendr works by taking a time series of annual values and fitting multiple linear regression models to recursively break the time series into segments that represent time periods with similar linear trends. The resulting segments can be used to describe change processes on the landscape. 

![workspace url](https://emapr.github.io/LT-GEE/imgs/segmentation.png)

By having the duration, magnitude, slope, etc of each segment, you can more easily detect change with simple rulesets.

![workspace url](https://emapr.github.io/LT-GEE/imgs/segment_attributes.png)

For example, a short, steep decline is likely to be a fast change, like harvest or fire. A long, shallow decline is likely to be a slower change, due to insects, disease, or drought. LandTrendr can also track periods of recovery, like a long shallow incline following a harvest or fire event. 

**For more information on LandTrendr, read [Chapter 4.5: Interpreting Annual Time Series with LandTrendr](https://docs.google.com/document/d/11oo1TuvXyEvReoYLDeTcoh16rEN90I8Bf5qp0KxVu7U/edit#heading=h.a480u2bjy8ur) in the [Cloud-Based Remote Sensing with Google Earth Engine ebook](https://www.eefabook.org/).**


### Example 
Below, see an example of data input to LandTrendr.

#### LandTrendr bands and parameters

Look at the bands included in the **actual** collection that LCMS uses for LandTrendr in Puerto Rico. Note that the path to the asset refers to the LCMS Google Cloud Project. Load the image collection and print the names of the bands. 

In [49]:
lcms_actual_lt_collection = ee.ImageCollection('projects/lcms-292214/assets/R8/PR_USVI/Base-Learners/LandTrendr-Collection-1984-2020')
print('All bands LCMS uses for LandTrendr:',lcms_actual_lt_collection.aggregate_histogram('band').keys().getInfo())

All bands LCMS uses for LandTrendr: ['NBR', 'NDMI', 'NDSI', 'NDVI', 'blue', 'brightness', 'green', 'greenness', 'nir', 'red', 'swir1', 'swir2', 'tcAngleBG', 'wetness']


#### Composites
For this lab, you'll 
    a) load the composites that were computed in the previous lab to 
    b) calculate fitted LandTrendr outputs that indicate the timing, duration, and magnitude of change-- among other outputs. 

Run the code block below. This code block will: 
* inspect the composites that you're bringing in
* get parameters from the composites that you will use as inputs to the LandTrendr algorithm
* calculate additional indices that you will use in LandTrendr

In [50]:
# Bring in composites and pull info from them
composites = ee.ImageCollection(export_composite_collection)

props = composites.first().toDictionary().getInfo()

startYear = props['startYear']
endYear = props['endYear']

startJulian = props['startJulian']
endJulian = props['endJulian']

proj = composites.first().projection().getInfo()

# Pull out the crs
# Depending on if a wkt or epsg format is used, it will be stored under a different key
if 'crs' not in proj.keys():
    crs = proj['wkt']
else:
    crs = proj['crs']
    
transform = proj['transform']
scale = None

studyArea = composites.first().geometry()

# Decompress composites by dividing by 10000 for optical bands and add indices
composites = composites.select(['blue','green','red','nir','swir1','swir2']).map(lambda img: img.divide(10000).float().copyProperties(img,['system:time_start']))
composites = composites.map(getImagesLib.simpleAddIndices)\
                      .map(getImagesLib.getTasseledCap)\
                      .map(getImagesLib.simpleAddTCAngles)


print('Done')

Done


#### Inspect
Now, inspect the composites. Run the code block below to add the composites to the map. You may need to click the button next to 'Composites' to view the layers on the map. Double-click the map to query the outputs. You should see a time series of values for each of the indices. 

In [53]:
# Run map
Map.clearMap()
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse,'Composites')

Map.centerObject(studyArea,9)
Map.turnOnInspector()
Map.view()

Adding layer: Composites
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [31/Aug/2023 17:04:46] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


## LandTrendr Inputs

### Input bands and band indices

We can use any or all of the bands in the composites for LandTrendr, but generally bands or band indices that use the NIR and SWIR portions of the electromagnetic spectrum are the most useful. To start, we will be running LandTrendr on only band index: NBR.

### LandTrendr parameters
The input parameters for the LandTrendr algorithm are described below. 

| Argument               | Type                    | Details                                                                                                                                                           |
|------------------------|-------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| timeSeries             | ImageCollection         | Yearly time-series from which to extract breakpoints. The first band is usedto find breakpoints, and all subsequent bands are fitted using those breakpoints.     |
| maxSegments            | Integer                 | Maximum number of segments to be fitted on the time series.                                                                                                       |
| spikeThreshold         | Float, default: 0.9     | Threshold for dampening the spikes (1.0 means no dampening).                                                                                                      |
| vertexCountOvershoot   | Integer, default: 3     | The initial model can overshoot the maxSegments + 1 vertices by this amount. Later, it will be pruned down to maxSegments + 1.                                    |
| preventOneYearRecovery | Boolean, default: False | Prevent segments that represent one year recoveries.                                                                                                              |
| recoveryThreshold      | Float, default: 0.25    | If a segment has a recovery rate faster than 1/recoveryThreshold (in years), then the segment is disallowed.                                                      |
| pvalThreshold          | Float, default: 0.1     | If the p-value of the fitted model exceeds this threshold, then the current model is discarded and another one is fitted using the Levenberg-Marquardt optimizer. |
| bestModelProportion    | Float, default: 0.75    | Allows models with more vertices to be chosen if their p-value is no more than (2 - bestModelProportion) times the p-value of the best model.                     |
| minObservationsNeeded  | Integer, default: 6     | Min observations needed to perform output fitting.                                                                                                                |

#### Parameter values
LCMS generally uses the default parameters for the GEE implementation of LandTrendr. Refer to the parameter values in the **GEE** column below. The table below is reproduced from [Kennedy et al. 2018](https://www.mdpi.com/2072-4292/10/5/691), which developed the GEE implementation of LandTrendr from the original IDL version. 

|       Parameter       |  IDL |   GEE   |                                                                                                        Comments                                                                                                       |
|:---------------------:|:----:|:-------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
|      maxSegments      |   6  |    6    |                                                                                                                                                                                                                       |
|     spikeThreshold    |  0.9 |   0.9   | Renamed from “desawtooth val”                                                                                                                                                                                         |
|  vertexCountOvershoot |   3  |    3    |                                                                                                                                                                                                                       |
|   recoveryThreshold   | 0.25 |   0.25  |                                                                                                                                                                                                                       |
|     pvalThreshold     | 0.05 |   0.05  |                                                                                                                                                                                                                       |
|  bestModelProportion  | 0.75 |   0.75  |                                                                                                                                                                                                                       |
| minObservationsNeeded |   6  |    6    | Renamed from “minneeded”                                                                                                                                                                                              |
|     Background_val    |   0  |    NA   | GEE uses a mask logic to avoid missing values caused by clouds, shadows, and missing imagery.                                                                                                                         |
|        Divisor        |  −1  |    NA   | Ensures that vegetation loss disturbance results in negative change in value when NBR is used as a spectral metric. In GEE, this must be handled outside of the segmentation algorithm.                               |
|       Kernelsize      |   1  | Dropped | Originally used together with skipfactor to save computational burden; no longer necessary.                                                                                                                           |
|       Skipfactor      |   1  | Dropped |                                                                                                                                                                                                                       |
|    Distweightfactor   |   2  | Dropped | Inadvertently hardwired in the IDL code, this parameter was hardwired in the GEE code to the value of 2.                                                                                                              |
|     Fix_doy_effect    |   1  | Dropped | Although correcting day-of-year trends was considered theoretically useful in the original LT implementation, in practice it has been found to distort time series values when change occurs and thus was eliminated. |

**Run the code block below to set the parameters.**

In [54]:
#Define landtrendr params
run_params = { 
  'maxSegments':            6,
  'spikeThreshold':         0.9,
  'vertexCountOvershoot':   3,
  'preventOneYearRecovery': False,
  'recoveryThreshold':      0.25,
  'pvalThreshold':          0.05,
  'bestModelProportion':    0.75,
  'minObservationsNeeded':  6
}

print('Done')

Done


## Run LandTrendr - single index
You will start by running LandTrendr on a single index in order to familiarize yourself with the process and the outputs. 

You will run LandTrendr using [Normalized Burn Ratio](https://www.usgs.gov/landsat-missions/landsat-normalized-burn-ratio) (NBR).  NBR is a ratio between the NIR and SWIR bands, and is sensitive to changes in moisture and vegetation cover. This will allow you to inspect changes on the landscape without requiring the computation time necessary for all of the bands and band indices in our composite.



In [55]:
# clear map
Map.clearMap()

# set up LandTrendr
test_band = 'NBR'
run_params['timeSeries'] = composites.select([test_band])
raw_LT = ee.Algorithms.TemporalSegmentation.LandTrendr(**run_params)

# Add to Map
Map.addLayer(raw_LT,{},'LT Raw {}'.format(test_band),True)

Map.turnOnInspector()
Map.view()

Adding layer: LT Raw NBR
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [31/Aug/2023 17:21:22] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


Double click on the output on the map to query it. 

Notice that this output is not immediately useful for change detection, or smoothing out a time series of composites. The default output is in array format, and has to be processed more in order for us to apply meaningful symbology on the map.


## Use LandTrendr for Change Detection
Using the outputs you just generated, you will now go through each step to take the raw LandTrendr output and create a basic change detection output.

At each step, you will view the results at a single pixel in order to inspect the changes on the array of outputs that you are manipulating. 

### Inspect raw LandTrendr outputs

Run the code block below to inspect the raw LandTrendr outputs.

In [62]:
# Provide an example location 
pt = ee.Geometry.Point([ -65.944 , 18.404])

# First, select the LandTrendr image array output band
lt_array = raw_LT.select(['LandTrendr'])

# Display output
display(g2p.imageArrayPixelToDataFrame(lt_array, pt,None,crs,transform, 'Raw LandTrendr Output - Single Pixel',\
                                       ['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
Years,1984.000000,1985.000000,1986.000000,1987.000000,1988.000000,1989.000000,1990.000000,1992.000000,1999.000000,2000.000000,2002.000000,2003.000000,2006.000000,2009.000000,2010.000000,2012.000000,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018.000000,2019.000000,2020.000000,2021.000000,2022.000000
Raw Input Values,0.732861,0.567021,0.772094,0.716129,0.765138,0.656260,0.674274,0.714593,0.100228,0.086324,0.098278,0.035483,0.148884,0.076102,0.068394,0.125019,0.277041,0.380944,0.452543,0.547757,0.615267,0.552308,0.558119,0.336950,0.426205,0.430944
Fitted Output Values,0.759414,0.730053,0.700691,0.671330,0.641969,0.612607,0.583246,0.524523,0.318994,0.289633,0.230910,0.201549,0.113465,0.025381,-0.003980,-0.062703,0.111118,0.284938,0.458759,0.632580,0.589873,0.547167,0.504460,0.461754,0.419048,0.376341
Vertex/non-vertex,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


This is the raw output from the LandTrendr algorithm. The array has 2 dimensions per pixel.

The rows correspond to:
- Years
- Raw input spectral values
- LandTrendr fitted output values
- Whether or not that year represents a vertex

From this output, you can begin to understand the format of the outputs, why they aren't immediately interpretable on a map, and how you might begin to manipulate them into more meaningulf formats. 

### Extract the vertices

The first step is to extract the vertices from the array. We only need the vertices at a pixel in order to track change-- we don't need the values for the interceding years.

Run the code block below to extract the vertices. You slice the array to extract the row indicating the vertices and use them as a mask to mask out non vertex values in the entire array.

In [63]:
# Slice the array to extract the row indicating the vertices
vertices = lt_array.arraySlice(0,3,4)
display(g2p.imageArrayPixelToDataFrame(vertices, pt, None,crs,transform,'Vertex mask row'))

# Use the vertex row as a mask to extract the values at the vertices
lt_array = lt_array.arrayMask(vertices)
display(g2p.imageArrayPixelToDataFrame(lt_array, pt, None,crs,transform,'Raw LandTrendr - Only Vertex Columns',\
                                                 ['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1


,0,1,2,3
Years,1984.000000,2012.000000,2016.000000,2022.000000
Raw Input Values,0.732861,0.125019,0.547757,0.430944
Fitted Output Values,0.759414,-0.062703,0.632580,0.376341
Vertex/non-vertex,1.000000,1.000000,1.000000,1.000000


### Calculate the difference between fitted vertex values
In order to perform change detection, you'll need to get the difference between fitted vertex values. 

You do this by slicing the array on an offset-- so that you can subtract adjacent values. 

Run the code block below to calculate the difference between the fitted vertex values. 

In [64]:
# In order to perform change detection, we'll need to get the difference between fitted vertex values
# We do this by slicing the first to the next to the last and then the second to the last and subtracting them
left = lt_array.arraySlice(1,0,-1)
right = lt_array.arraySlice(1,1,None)
diff  = left.subtract(right)

display(g2p.imageArrayPixelToDataFrame(left, pt, None,crs,transform,'Left Slice',\
                                       ['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))
display(g2p.imageArrayPixelToDataFrame(right, pt,None,crs,transform, \
                                       'Right Slice',['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))
display(g2p.imageArrayPixelToDataFrame(diff, pt, None,crs,transform,\
                                       'Left Minus Right',['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))

,0,1,2
Years,1984.000000,2012.000000,2016.000000
Raw Input Values,0.732861,0.125019,0.547757
Fitted Output Values,0.759414,-0.062703,0.632580
Vertex/non-vertex,1.000000,1.000000,1.000000


,0,1,2
Years,2012.000000,2016.000000,2022.000000
Raw Input Values,0.125019,0.547757,0.430944
Fitted Output Values,-0.062703,0.632580,0.376341
Vertex/non-vertex,1.000000,1.000000,1.000000


,0,1,2
Years,-28.000000,-4.000000,-6.000000
Raw Input Values,0.607841,-0.422738,0.116814
Fitted Output Values,0.822117,-0.695283,0.256238
Vertex/non-vertex,0.000000,0.000000,0.000000


### Combine difference values with years
You then slice the right-hand years and the fitted vertex values difference and combine them.

Run the code block below to create an array that has a value for each vertex year and magnitude of difference. 

In [68]:
# Slice right-hand years and difference values
years = right.arraySlice(0,0,1)
mag = diff.arraySlice(0,2,3)
display(g2p.imageArrayPixelToDataFrame(years, pt, None,crs,transform,'Years'))
display(g2p.imageArrayPixelToDataFrame(mag, pt, None,crs,transform,'Magnitude'))

# Combine
forSorting = years.arrayCat(mag,0)
display(g2p.imageArrayPixelToDataFrame(forSorting, pt, None,crs,transform,'Year + Magnitude Array'))

,0,1,2
0,2012,2016,2022


,0,1,2
0,0.822117,-0.695283,0.256238


,0,1,2
0,2012.000000,2016.000000,2022.000000
1,0.822117,-0.695283,0.256238


### Sort array based on change of interest

We can then sort this array to display the change we are most interested in. For example, we can extract the highest magnitude change, the most recent change, etc. 

In the example below, the sort row will be the magnitude. Thus, the output will be the highest severity change.


In [69]:
# Sort on magnitude
sorted = forSorting.arraySort(forSorting.arraySlice(0,1,2))
display(g2p.imageArrayPixelToDataFrame(sorted, pt, None,crs,transform,'Array Sorted by the Second Row (magnitude of change)'))

# Slice off the year and magnitude of the highest magnitude change
highest_mag_change_array = sorted.arraySlice(1,0,1)
display(g2p.imageArrayPixelToDataFrame(highest_mag_change_array, pt, None,crs,transform,'Highest Mag Change (year and magnitude)'))

,0,1,2
0,2016.000000,2022.000000,2012.000000
1,-0.695283,0.256238,0.822117


,0
0,2016.000000
1,-0.695283


### Convert arrays into images

The final step is to convert the array output into an image. You've seen how array manipulation works on one pixel. You can now "flatten" the two-dimensional array into a one-dimensional image. 

Applying a change threshold allows you to determine what severity of change is flagged as loss. All change that is lower than this magnitude will be masked from the map. 

Run the code block below to flatten the array into an image and add it to the map. 

In [70]:
# First, choose a change threshold
# Any changes more negative than this value will be flagged as loss
change_threshold = -0.15

# Convert the sorted array image into a 2-band image
highest_mag_change = highest_mag_change_array.arrayProject([0]).arrayFlatten([['yr','mag']])

# Mask out any change 
highest_mag_change = highest_mag_change.updateMask(highest_mag_change.select(['mag']).lte(change_threshold))

# Pull the loss magnitude palette and flip the color order
lossMagPalette = changeDetectionLib.lossMagPalette.split(',')
lossMagPalette.reverse()

# Set up map
Map.clearMap()
Map.addLayer(forSorting,{},'Array for Sorting',False)
Map.addLayer(sorted,{},'Sorted Array',False)
Map.addLayer(highest_mag_change.select(['mag']),{'min':-0.8,'max':-0.15,'palette':lossMagPalette},'Loss Magnitude')
Map.addLayer(highest_mag_change.select(['yr']),{'min':startYear,'max':endYear,'palette':changeDetectionLib.lossYearPalette},'Loss Year')
Map.turnOnInspector()
Map.view()

Adding layer: Array for Sorting
Adding layer: Sorted Array
Adding layer: Loss Magnitude
Adding layer: Loss Year
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [31/Aug/2023 17:47:28] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


### Inspect the output
View the output on the map. Turn the layers off and on to view the Loss Year and Loss Magnitude rasters. Double click on a pixel to query it, and see the original array values to understand how they correspond to the end output. 

## Run LandTrendr - on all bands

Now that you understand LandTrendr parameters and outputs, the next step is to run LandTrendr on all bands. This is a more realistic output. After some additional processing, you will use this output with the LCMS model in Modules 4 and 5. 

Note that the code below is the same as the code you used to run LandTrendr over a single band. But here, you are applying LandTrendr over each band, and you are not displaying the outputs of the array manipulation at each step. 

Run the code block below to compute and export LandTrendr output arrays for each of the bands listed in `bandNames`. 

In [ ]:
# We will run LandTrendr for each band
#Clear the map in case it has been populated with layers/commands earlier
Map.clearMap()

# We can use any/all of the bands, but generally bands that use nir and swir are most useful
bandNames = ['red','nir','swir1','swir2','NBR','NDVI','brightness','greenness','wetness']

#Run LANDTRENDR
for bandName in bandNames:

    # Select the band and run LandTrendr
    run_params['timeSeries'] = composites.select([bandName])
    rawLT = ee.Algorithms.TemporalSegmentation.LandTrendr(**run_params)

    Map.addLayer(rawLT,{},'LT Raw {}'.format(bandName),False)

    # Notice the raw LandTrendr output is in GEE's image array format
    # We'll need to manipulate the raw output a bit to save on storage space

    # Mask out non vertex values to use less storage space
    ltArray = rawLT.select(['LandTrendr'])
    rmse = rawLT.select(['rmse'])
    vertices = ltArray.arraySlice(0,3,4)
    ltArray = ltArray.arrayMask(vertices)

    # Mask out all but the year and vertex fited values (get rid of the raw and vertex rows)
    ltArray = ltArray.arrayMask(ee.Image(ee.Array([[1],[0],[1],[0]])))
    rawLTForExport=ltArray.addBands(rmse)
    Map.addLayer(rawLTForExport,{},'LT Vertex Values Only {}'.format(bandName),False)

    # Show how the compressed vertex-only values can be decompressed later
    decompressedC = changeDetectionLib.simpleLTFit(ltArray,startYear,endYear,bandName,True,run_params['maxSegments'])
    Map.addLayer(decompressedC,{'bands':'{}_LT_fitted'.format(bandName),'min':0.2,'max':0.8},'Decompressed LT Output {}'.format(bandName),False)

    # Join the raw and fitted values
    fitted = decompressedC.select(['{}_LT_fitted'.format(bandName)])
    ltJoined = getImagesLib.joinCollections(composites.select([bandName]),fitted)
    Map.addLayer(ltJoined,{'bands':'{}_LT_fitted'.format(bandName),'min':0.2,'max':1,'palette':'D80,080'},'Raw and LT Fitted {}'.format(bandName),True)

    # Export LT array image
    # Set some properties that will be uses later
    rawLTForExport = rawLTForExport.set({'startYear':startYear,
                                          'endYear':endYear,
                                          'startJulian':startJulian,
                                          'endJulian':endJulian,
                                          'band':bandName})
    rawLTForExport =rawLTForExport.set(run_params)
    exportName = 'LT_Raw_{}_yrs{}-{}_jds{}-{}'.format(bandName,startYear,endYear,startJulian,endJulian)
    exportPath = export_landTrendr_collection + '/'+ exportName
    # Export output
    getImagesLib.exportToAssetWrapper(rawLTForExport,exportName,exportPath,{'.default':'sample'},studyArea,scale,crs,transform,overwrite=False)

Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

If you'd like to track the status of export tasks, use the code below. 

In [ ]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(export_landTrendr_collection, type = 'imageCollection')

print('done')

> Note: The geeViz library also provides wrapper functions for array processing and running LandTrendr. You can use the [`changeDetectionLib.convertToLossGain()`](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/changeDetectionLib.py#L778C5-L778C22) function in the geeViz library to perform array processing. You can also use the [`changeDetectionLib.simpleLANDTRENDR`](https://github.com/gee-community/geeViz/blob/27a0c5d8a0a9c9623e67599bf06448d64b481c56/changeDetectionLib.py#L344) to run LandTrendr. Check out examples and documentation in the [geeViz/examples](https://github.com/gee-community/geeViz/blob/master/examples/LANDTRENDRViz.py) repository.





### Convert LandTrendr array into time series - for input into LCMS

While we can use the LandTrendr output for change detection, LCMS uses it as inputs to change detection, land cover, and land use models. Next, you will conver the raw LandTrendr array image asset into a time series of annual fitted, segment duration, segment magnitude of change, and slope values.

This processing relies on the [`changeDetectionLib.batchSimpleLTFit`](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/changeDetectionLib.py#L565) function. This function converts stacked outputs into a collection of fitted, annual outputs: e.g., magnitude of change, slope of change, duration of change for each year.

While the fitted LandTrendr value is generally of most importance to our models, LandTrendr segment duration, slope, and magnitude of change can also help our models.

Run the code block below to extract the fitted annual outputs and view them on the map.

In [46]:
# Load raw LandTrendr outputs
lt_asset = ee.ImageCollection(f'{pre_baked_path_root}/lcms-training_module-3_landTrendr')

# Convert into fitted, annual outputs: e.g., magnitude of change, slope of change, duration of change for each year
lt_fit = changeDetectionLib.batchSimpleLTFit(lt_asset,startYear,endYear,None,bandPropertyName='band',arrayMode=True)

#Add to the map
Map.clearMap()
Map.addLayer(lt_fit,{'bands':'swir2_LT_fitted,nir_LT_fitted,red_LT_fitted','min':0.15,'max':0.6},'LandTrendr All Predictors Time Series')


Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: LandTrendr All Predictors Time Series
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


127.0.0.1 - - [31/Aug/2023 15:10:38] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


### Inspect
Double-click the map to query values and see actual and fitted values for indices over time. Vizualize image collection that is used as predictors in LCMS models. When you double-click the output on the map, notice the different values available to the models

## Example: LandTrendr reduces noise in original composite time series 

The best way of understanding how LandTrendr contributes to reducing noise in the original composite time series is to visualize the LandTrendr outputs and the composite side-by-side. 

The below example takes the fitted values from LandTrendr and shows them along with the original composites. Notice many holes are now filled in by LandTrendr. In general, LandTrendr reduces the amount of noise in the time series. There is a risk, however, of fitting too much and omitting changes such as those seen in 2017 for Hurricane Maria.

Run the code block below to compare the fitted time series against the input composites. 

Click on the map, inspect the fitted time series, and watch the composite timelapse and the Landtrendr timelapse.

In [71]:
# Visualize fitted landTrendr composites
fitted_bns = lt_fit.select(['.*_fitted']).first().bandNames()
out_bns = fitted_bns.map(lambda bn: ee.String(bn).split('_').get(0))

# Give same names as composites
lt_synth = lt_fit.select(fitted_bns,out_bns)

# Clear Map
Map.clearMap()

# Visualize raw and LandTrendr fitted composites
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse,'Raw Composite Timelapse')
Map.addTimeLapse(lt_synth,getImagesLib.vizParamsFalse,'Fitted LandTrendr Composite Timelapse')

# Join the raw and fitted values
ltJoined = getImagesLib.joinCollections(composites.select(bandNames),lt_fit.select(['.*_fitted']))

# Add to Map
Map.addLayer(ltJoined,{'min':0.2,'max':1},'Raw and LT Fitted',True)

Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: Raw Composite Timelapse
Adding layer: Fitted LandTrendr Composite Timelapse
Adding layer: Raw and LT Fitted
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [31/Aug/2023 18:10:56] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


### Challenge: 

Calculate LandTrendr stats for NDVI and process arrays into most recent change year.

**For Qwiklabs users**, this will be assessed for completion in the Activity Tracking portion of Module 3.

In [ ]:
# Insert challenge code here

### Congratulations! You're done with module 3.1.

Other GeeViz LandTrendr examples: 
- https://github.com/redcastle-resources/geeViz/blob/master/examples/LANDTRENDRViz.py
- https://github.com/redcastle-resources/geeViz/blob/master/examples/LANDTRENDRWrapper.py
- https://github.com/redcastle-resources/geeViz/blob/master/examples/LANDTRENDRWrapperNotebook.ipynb


LandTrendr fitted data will be used as inputs to LCMS in subsequent modules.